# Development of machine learning models to process Electronic Health Records – Explainable Models

### Imputation Notebook
Lok Hang Toby Lee (2431180L)

# Imputation

----------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

# SET YOUR PATH FOR RESOURCES FILE HERE
resources_path = "C:/Users/USER/OneDrive/GU/Year 4/FYP\MIMIC-III-ML/data/resources"
data_path = "C:/Users/USER/OneDrive/GU/Year 4/FYP/MIMIC-III-ML/data/raw"

### Extract the variables and count the missing data

In [2]:
# Read in the data and only select the continuous variables:
config = json.load(open(resources_path + '/discretizer_config.json', 'r'))
is_categorical = config['is_categorical_channel']
categorical_var = [key for key, value in config['is_categorical_channel'].items() if value][1:]
patient_identifiers = ['icustay_id', 'subject_id', 'hadm_id', 'hours_in']
data = pd.read_csv(data_path+'/mimic_timeseries_data_not_imputed.csv')

# Filter out subjects with length of stay < 48 hours for the in-hospital mortality task:
y = pd.read_hdf(data_path+'/vitals_hourly_data_preprocessed.h5', 'Y')['los']
indices_to_remove = []
for i, row in y.iteritems():
    if row < 48:
        indices_to_remove.append(i)
        
data = data.reset_index().set_index('icustay_id').drop(indices_to_remove, axis = 0)
y = y.drop(indices_to_remove, axis = 0)

# Extract only the first 48 hours:
data = data[data['hours_in'] < 48].reset_index().set_index(['icustay_id', 'subject_id', 'hadm_id', 'hours_in']).drop('index', axis = 1)

In [3]:
data

Unnamed: 0  Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                                         
200003     27513      163557  0                  0                 49.000000   
                              1                  1                 52.000000   
                              2                  2                 52.333333   
                              3                  3                 60.500000   
                              4                  4                 61.000000   
...                                            ...                       ...   
299995     28775      134959  43           2162994                 82.000000   
                              44           2162995                       NaN   
                              45           2162996                       NaN   
                              46           2162997                 65.000000   
                              47           2162998                       NaN   

                                        Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                              NaN   
                              1                              NaN   
                              2                              NaN   
                              3                              NaN   
                              4                              NaN   
...                                                          ...   
299995     28775      134959  43                             NaN   
                              44                             NaN   
                              45                             NaN   
                              46                             NaN   
                              47                             NaN   

                                        Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                   
200003     27513      163557  0                                    4.0   
                              1                                    NaN   
                              2                                    NaN   
                              3                                    NaN   
                              4                                    NaN   
...                                                                ...   
299995     28775      134959  43                                   NaN   
                              44                                   NaN   
                              45                                   NaN   
                              46                                   4.0   
                              47                                   NaN   

                                        Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                      
200003     27513      163557  0                                       6.0   
                              1                                       NaN   
                              2                                       NaN   
                              3                                       NaN   
                              4                                       NaN   
...                                                                   ...   
299995     28775      134959  43                                      NaN   
                              44                                      NaN   
                              45                                      NaN   
                              46                                      6.0   
                              47                                      NaN   

                                        Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                             
200003     27

In [4]:
print('Number of samples: ', data.shape[0])
print('Number of ICU stays: ', len(data.groupby('icustay_id')))

non_nans_per_predictor = np.array(list(data.groupby('icustay_id').apply(lambda x: np.count_nonzero(~np.isnan(x.values[:, 4:]), 
                                                                                           axis = 0))))
total_missing_predictors = 0
for subject in non_nans_per_predictor:
    for predictor in subject:
        if predictor == 0:
            total_missing_predictors += 1
print('Total number of predictors that are missing at every hour (per subject): ', total_missing_predictors)

Number of samples:  843312
Number of ICU stays:  17569
Total number of predictors that are missing at every hour (per subject):  29010


# Mean and joint imputation

In [5]:
# Initialize matrix to store the results of different imputation methods:
data = data.drop('Unnamed: 0', axis = 1)
original_data = pd.DataFrame(index = data.index, columns = data.columns)
results_mean_imp = pd.DataFrame(index = data.index, columns = data.columns)
results_joint_imp = pd.DataFrame(index = data.index, columns = data.columns)

In [6]:
from sklearn.impute import KNNImputer

counter = 0
n_folds = 100
unique_icustays = np.unique(data.index.get_level_values('icustay_id'))
kf = KFold(n_splits=n_folds, random_state=0, shuffle=True)
for train_index, test_index in kf.split(unique_icustays):
    
    print('Imputing fold ', counter + 1, '/', n_folds, end = '\r')
    counter += 1
  
    train_index = unique_icustays[train_index]
    test_index = unique_icustays[test_index]

    data_train = data.iloc[data.index.get_level_values('icustay_id').isin(train_index)]
    data_test = data.iloc[data.index.get_level_values('icustay_id').isin(test_index)]

    for i in range(len(data.columns)): # round categorical to closest category
        if data.columns[i] in categorical_var:
            data_test.loc[:, data.columns[i]] = np.round(data_test.loc[:, data.columns[i]])
    original_data.loc[data.index.get_level_values('icustay_id').isin(test_index), :] = data_test
    
    # Replace the test data with NaNs so we can learn the values:
    data_test = np.empty(data_test.shape)
    data_test[:] = np.nan
    
    # Mean imputation:
    mean_imp = np.nanmean(data_train, axis = 0)
    for i in range(len(mean_imp)): # round categorical to closest category
        if data.columns[i] in categorical_var:
            mean_imp[i] = np.round(mean_imp[i])  
    results_mean_imp.loc[data.index.get_level_values('icustay_id').isin(test_index), :] = mean_imp
    
    # Joint imputation: first extract the mu and covariance, then sample from the Gaussian distribution:    
    mu = np.nanmean(data_train.values, axis = 0) #computes the mean along the specified axis ignoring all NaNs.
    data_train_no_nans = data_train.values
    for i in range(data_train.values.shape[1]):
        data_train_no_nans[:, i] = np.nan_to_num(data_train.values[:, i], nan = mu[i])
    data_train_no_nans = data_train_no_nans.T
    sigma = np.cov(data_train_no_nans)
    sample = np.random.multivariate_normal(mu, sigma, size = len(unique_icustays))
    sample = np.mean(sample, axis = 0)
    for i in range(len(sample)): # round categorical to closest category
        if data.columns[i] in categorical_var:
            sample[i] = np.round(sample[i])
    results_joint_imp.loc[data.index.get_level_values('icustay_id').isin(test_index), :] = sample


# KNN imputation

In [18]:
results_knn_imp = pd.DataFrame(index = data.index, columns = data.columns)


In [33]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(data)
scaled_data= scaler.transform(data)
scaled_data=np.array_split(scaled_data,8)

In [28]:

# KNN imputation
knn_imputed_results=[]
counter=0
for i in range(len(scaled_data)):
    print('Imputing split ', counter + 1, '...')
    counter += 1
    knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
    knn_imp = knn_imputer.fit_transform(scaled_data[i])
    knn_imputed_results.append(knn_imp)


Imputing split  1 ...
Imputing split  2 ...
Imputing split  3 ...
Imputing split  4 ...
Imputing split  5 ...
Imputing split  6 ...
Imputing split  7 ...
Imputing split  8 ...


In [48]:
knn_results=np.concatenate(knn_imputed_results)
knn_results=scaler.inverse_transform(knn_results)
knn_imputed_data = pd.DataFrame(index = data.index, columns = data.columns)
knn_imputed_data.loc[:,:]=knn_results

# MICE

In [10]:
import miceforest as mf

mf.ampute_data(data,perc=0.25,random_state=1991)

# Create kernels. 
kernel = mf.ImputationKernel(
  data=data,
  save_all_iterations=True,
  random_state=1991
)

# Run the MICE algorithm for 3 iterations on each of the datasets
kernel.mice(3,verbose=True)

Dataset 0
1  | Heart Rate | Respiratory rate | Systolic blood pressure | Diastolic blood pressure | Mean blood pressure | Oxygen saturation | Weight | Temperature | Glascow coma scale eye opening | Glascow coma scale verbal response | Glascow coma scale motor response | Glucose | Glascow coma scale total | pH | Fraction inspired oxygen | Height
2  | Heart Rate | Respiratory rate | Systolic blood pressure | Diastolic blood pressure | Mean blood pressure | Oxygen saturation | Weight | Temperature | Glascow coma scale eye opening | Glascow coma scale verbal response | Glascow coma scale motor response | Glucose | Glascow coma scale total | pH | Fraction inspired oxygen | Height
3  | Heart Rate | Respiratory rate | Systolic blood pressure | Diastolic blood pressure | Mean blood pressure | Oxygen saturation | Weight | Temperature | Glascow coma scale eye opening | Glascow coma scale verbal response | Glascow coma scale motor response | Glucose | Glascow coma scale total | pH | Fraction insp

In [11]:
mice_imputed_data = kernel.complete_data(4)

In [12]:
mice_imputed_data

Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                        49.000000   
                              1                        52.000000   
                              2                        52.333333   
                              3                        60.500000   
                              4                        61.000000   
...                                                          ...   
299995     28775      134959  43                       82.000000   
                              44                       53.000000   
                              45                       96.000000   
                              46                       65.000000   
                              47                       54.000000   

                                        Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                             1.00   
                              1                             0.40   
                              2                             0.40   
                              3                             0.50   
                              4                             0.50   
...                                                          ...   
299995     28775      134959  43                            0.70   
                              44                            0.30   
                              45                            0.50   
                              46                            0.28   
                              47                            0.40   

                                        Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                   
200003     27513      163557  0                                    4.0   
                              1                                    1.0   
                              2                                    4.0   
                              3                                    4.0   
                              4                                    2.0   
...                                                                ...   
299995     28775      134959  43                                   4.0   
                              44                                   3.0   
                              45                                   2.0   
                              46                                   4.0   
                              47                                   1.0   

                                        Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                      
200003     27513      163557  0                                       6.0   
                              1                                       4.0   
                              2                                       6.0   
                              3                                       6.0   
                              4                                       5.0   
...                                                                   ...   
299995     28775      134959  43                                      6.0   
                              44                                      6.0   
                              45                                      4.0   
                              46                                      6.0   
                              47                                      4.0   

                                        Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                             15.0   
                              1                              6.0   
                              2  

In [54]:
mean_imputed_data = original_data.fillna(results_mean_imp)
joint_imputed_data = original_data.fillna(results_joint_imp)

mean_imputed_data.to_csv(data_path+'/imputed_data/mean_imputed.csv')
joint_imputed_data.to_csv(data_path+'/imputed_data/joint_imputed.csv')
mice_imputed_data.to_csv(data_path+'/imputed_data/mice_imputed.csv')
knn_imputed_data.to_csv(data_path+'/imputed_data/knn_imputed.csv')

In [50]:
mean_imputed_data

Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                            49.0   
                              1                            52.0   
                              2                       52.333333   
                              3                            60.5   
                              4                            61.0   
...                                                         ...   
299995     28775      134959  43                           82.0   
                              44                      60.109355   
                              45                      60.109355   
                              46                           65.0   
                              47                      60.109355   

                                       Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                        0.546215   
                              1                        0.546215   
                              2                        0.546215   
                              3                        0.546215   
                              4                        0.546215   
...                                                         ...   
299995     28775      134959  43                       0.546321   
                              44                       0.546321   
                              45                       0.546321   
                              46                       0.546321   
                              47                       0.546321   

                                       Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                  
200003     27513      163557  0                                   4.0   
                              1                                   3.0   
                              2                                   3.0   
                              3                                   3.0   
                              4                                   3.0   
...                                                               ...   
299995     28775      134959  43                                  3.0   
                              44                                  3.0   
                              45                                  3.0   
                              46                                  4.0   
                              47                                  3.0   

                                       Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                     
200003     27513      163557  0                                      6.0   
                              1                                      5.0   
                              2                                      5.0   
                              3                                      5.0   
                              4                                      5.0   
...                                                                  ...   
299995     28775      134959  43                                     5.0   
                              44                                     5.0   
                              45                                     5.0   
                              46                                     6.0   
                              47                                     5.0   

                                       Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                            15.0   
                              1                            12.0   
                              2                            12.0   
                     

In [51]:
joint_imputed_data

Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                            49.0   
                              1                            52.0   
                              2                       52.333333   
                              3                            60.5   
                              4                            61.0   
...                                                         ...   
299995     28775      134959  43                           82.0   
                              44                      60.258372   
                              45                      60.258372   
                              46                           65.0   
                              47                      60.258372   

                                       Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                        0.546575   
                              1                        0.546575   
                              2                        0.546575   
                              3                        0.546575   
                              4                        0.546575   
...                                                         ...   
299995     28775      134959  43                       0.546112   
                              44                       0.546112   
                              45                       0.546112   
                              46                       0.546112   
                              47                       0.546112   

                                       Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                  
200003     27513      163557  0                                   4.0   
                              1                                   3.0   
                              2                                   3.0   
                              3                                   3.0   
                              4                                   3.0   
...                                                               ...   
299995     28775      134959  43                                  3.0   
                              44                                  3.0   
                              45                                  3.0   
                              46                                  4.0   
                              47                                  3.0   

                                       Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                     
200003     27513      163557  0                                      6.0   
                              1                                      5.0   
                              2                                      5.0   
                              3                                      5.0   
                              4                                      5.0   
...                                                                  ...   
299995     28775      134959  43                                     5.0   
                              44                                     5.0   
                              45                                     5.0   
                              46                                     6.0   
                              47                                     5.0   

                                       Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                            15.0   
                              1                            12.0   
                              2                            12.0   
                     

In [52]:
mice_imputed_data

Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                        49.000000   
                              1                        52.000000   
                              2                        52.333333   
                              3                        60.500000   
                              4                        61.000000   
...                                                          ...   
299995     28775      134959  43                       82.000000   
                              44                       53.000000   
                              45                       96.000000   
                              46                       65.000000   
                              47                       54.000000   

                                        Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                             1.00   
                              1                             0.40   
                              2                             0.40   
                              3                             0.50   
                              4                             0.50   
...                                                          ...   
299995     28775      134959  43                            0.70   
                              44                            0.30   
                              45                            0.50   
                              46                            0.28   
                              47                            0.40   

                                        Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                   
200003     27513      163557  0                                    4.0   
                              1                                    1.0   
                              2                                    4.0   
                              3                                    4.0   
                              4                                    2.0   
...                                                                ...   
299995     28775      134959  43                                   4.0   
                              44                                   3.0   
                              45                                   2.0   
                              46                                   4.0   
                              47                                   1.0   

                                        Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                      
200003     27513      163557  0                                       6.0   
                              1                                       4.0   
                              2                                       6.0   
                              3                                       6.0   
                              4                                       5.0   
...                                                                   ...   
299995     28775      134959  43                                      6.0   
                              44                                      6.0   
                              45                                      4.0   
                              46                                      6.0   
                              47                                      4.0   

                                        Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                             15.0   
                              1                              6.0   
                              2  

In [53]:
knn_imputed_data

Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                            49.0   
                              1                            52.0   
                              2                       52.333333   
                              3                            60.5   
                              4                            61.0   
...                                                         ...   
299995     28775      134959  43                           82.0   
                              44                           50.2   
                              45                           80.2   
                              46                           65.0   
                              47                      60.127424   

                                       Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                            0.66   
                              1                            0.57   
                              2                            0.53   
                              3                            0.63   
                              4                            0.73   
...                                                         ...   
299995     28775      134959  43                            0.5   
                              44                           0.64   
                              45                            0.5   
                              46                           0.56   
                              47                       0.546201   

                                       Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                  
200003     27513      163557  0                                   4.0   
                              1                                   3.4   
                              2                                   3.4   
                              3                                   4.0   
                              4                                   4.0   
...                                                               ...   
299995     28775      134959  43                                  3.8   
                              44                                  4.0   
                              45                                  3.0   
                              46                                  4.0   
                              47                             3.237221   

                                       Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                     
200003     27513      163557  0                                      6.0   
                              1                                      5.0   
                              2                                      5.4   
                              3                                      6.0   
                              4                                      5.8   
...                                                                  ...   
299995     28775      134959  43                                     6.0   
                              44                                     4.6   
                              45                                     4.6   
                              46                                     6.0   
                              47                                5.340042   

                                       Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                            
200003     27513      163557  0                            15.0   
                              1                            12.6   
                              2                            13.2   
                     

In [17]:
data

Diastolic blood pressure  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                        49.000000   
                              1                        52.000000   
                              2                        52.333333   
                              3                        60.500000   
                              4                        61.000000   
...                                                          ...   
299995     28775      134959  43                       82.000000   
                              44                             NaN   
                              45                             NaN   
                              46                       65.000000   
                              47                             NaN   

                                        Fraction inspired oxygen  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                              NaN   
                              1                              NaN   
                              2                              NaN   
                              3                              NaN   
                              4                              NaN   
...                                                          ...   
299995     28775      134959  43                             NaN   
                              44                             NaN   
                              45                             NaN   
                              46                             NaN   
                              47                             NaN   

                                        Glascow coma scale eye opening  \
icustay_id subject_id hadm_id hours_in                                   
200003     27513      163557  0                                    4.0   
                              1                                    NaN   
                              2                                    NaN   
                              3                                    NaN   
                              4                                    NaN   
...                                                                ...   
299995     28775      134959  43                                   NaN   
                              44                                   NaN   
                              45                                   NaN   
                              46                                   4.0   
                              47                                   NaN   

                                        Glascow coma scale motor response  \
icustay_id subject_id hadm_id hours_in                                      
200003     27513      163557  0                                       6.0   
                              1                                       NaN   
                              2                                       NaN   
                              3                                       NaN   
                              4                                       NaN   
...                                                                   ...   
299995     28775      134959  43                                      NaN   
                              44                                      NaN   
                              45                                      NaN   
                              46                                      6.0   
                              47                                      NaN   

                                        Glascow coma scale total  \
icustay_id subject_id hadm_id hours_in                             
200003     27513      163557  0                             15.0   
                              1                              NaN   
                              2  